In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import models

In [ ]:
data_student = "/content/drive/MyDrive/KD2_Predict_ArapUbi/Data/Data for Student/"

path_model = "/content/drive/MyDrive/KD2_Predict_ArapUbi/Model/"
path_result = "/content/drive/MyDrive/KD2_Predict_ArapUbi/Result/"

In [ ]:

def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)
num_folds = 5
TIME_STEPS = 33
INPUT_SIZE = 300

**Load data for Student training**

In [ ]:
# Load Traindata
import pandas as pd
file_train_student = "train_data_31_Arathailiana_1_1.csv"
df_train_student =pd.read_csv(data_student+file_train_student, delimiter= ',')

texts_train_student =[] #PTMsequend kmer
for i in df_train_student['Sequence']:
  temp = ProSentence(i,k)
  texts_train_student.append(temp)
df_train_student['k_mer'] =texts_train_student
train_sequences_student = []
for each in texts_train_student:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences_student.append(each_index_list)
# Tokenizer train data
data_token_student = []
for i in df_train_student['k_mer']:
   data_token_student.append(i.split())
MAX_SEQUENCE_LENGTH = len(data_token_student[1])

Xtrain_student = pad_sequences(train_sequences_student, maxlen=MAX_SEQUENCE_LENGTH)
ytrain_student = np.array(df_train_student['Label'])
ytrain_student = np.array(ytrain_student)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
ytrain_student = lb.fit_transform(ytrain_student)
ytrain_student = to_categorical(ytrain_student)
ytrain_student.shape


(3064, 2)

In [ ]:
# load test data for student
file_test_student ="test_data_31_Arathailiana_1_1.csv"
df_test_student =pd.read_csv(data_student+file_test_student,delimiter= ',')
text_test_student =[] #PTMsequend kmer
for i in df_test_student['Sequence']:
  temp = ProSentence(i,k)
  text_test_student.append(temp)
df_test_student['k_mer'] =text_test_student

test_sequences_student = []
for each in text_test_student:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    test_sequences_student.append(each_index_list)

Xtest_student = pad_sequences(test_sequences_student, maxlen=MAX_SEQUENCE_LENGTH)
ytest_student = np.array(df_test_student['Label'])
# perform one-hot encoding on the labels
ytest_student = np.array(ytest_student)
lb = LabelBinarizer()
ytest_student= lb.fit_transform(ytest_student)
ytest_student = to_categorical(ytest_student)
ytest_student.shape

(1022, 2)

**Define Distillation Knowlege**

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # Compute scaled distillation loss from https://arxiv.org/abs/1503.02531
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results
teacher = keras.Sequential(
    [
        layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True),
        #layers.Embedding(len(embedding_matrix), EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH_teacher,trainable=True),
        layers.Bidirectional(LSTM(units=32,batch_input_shape=(None,TIME_STEPS, INPUT_SIZE),return_sequences=True)),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(128),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        layers.Dense(2),
        layers.Activation('softmax'),
    ],
    name="teacher",
)

teacher.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[
        "categorical_accuracy",
        "AUC",

    ],
    optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
)
student = keras.Sequential(
    [
        layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True),
        #layers.Embedding(len(embedding_matrix), EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH_student,trainable=True),
        layers.Bidirectional(LSTM(units=16,batch_input_shape=(None,TIME_STEPS, INPUT_SIZE),return_sequences=True)),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(128),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        layers.Dense(2),
        layers.Activation('softmax'),
    ],
    name="student",
)
student_scratch = keras.models.clone_model(student)

In [ ]:
checkpoint = EarlyStopping(monitor='val_loss',
            min_delta=0,
            patience=3,
            verbose=1, mode='auto')
result_test =path_result +"Result_KD2_1gram_id.txt"


In [ ]:
# Independent test Knowlege distillation model
#Load model teacher for train student
teacher_model = models.load_model(path_model+'teacher_KD2_1gram.h5')
distiller = Distiller(student=student, teacher=teacher_model)
distiller.compile(
      student_loss_fn= keras.losses.CategoricalCrossentropy(from_logits=True),
      #student_loss_fn= keras.losses.BinaryCrossentropy(from_logits=True),
      optimizer=keras.optimizers.AdamW(learning_rate = 0.0001),
      metrics=[
              "categorical_accuracy",
              "AUC",
              ], # , f1
      distillation_loss_fn=keras.losses.KLDivergence(),
      alpha=0.1,
      temperature=10
      )

history_KD = distiller.fit(Xtrain_student,ytrain_student, batch_size=16,epochs=50)#, validation_data =(Xtest_student, ytest_student), callbacks=[checkpoint],verbose=1)
result2 = distiller.evaluate(Xtest_student, ytest_student)#, validation_data=(X_test,Y_test), shuffle = True,callbacks=[checkpoint],verbose=1)
f = open(result_test, 'a+', encoding='UTF-8')
f.write("\n Independent test  \n ")
s = str(result2)
f.write(s)
f.close()

ypred = student.predict(Xtest_student)
ypred =np.argmax(ypred,axis =1)

ytest_true =  np.argmax(ytest_student,axis =1)

result22 = confusion_matrix(ytest_true,ypred)
print(result22)
f = open(result_test, 'a+', encoding='UTF-8')
f.write("\n confusion_matrix \n ")
s = str(result22)
f.write(s)
f.close()


Epoch 1/50
192/192 [==============================] - 7s 15ms/step - categorical_accuracy: 0.7954 - auc: 0.8592 - student_loss: 0.4722 - distillation_loss: 0.0262
Epoch 2/50
192/192 [==============================] - 3s 14ms/step - categorical_accuracy: 0.7967 - auc: 0.8752 - student_loss: 0.4485 - distillation_loss: 0.0241
Epoch 3/50
192/192 [==============================] - 3s 14ms/step - categorical_accuracy: 0.8107 - auc: 0.8928 - student_loss: 0.4237 - distillation_loss: 0.0213
Epoch 4/50
192/192 [==============================] - 3s 14ms/step - categorical_accuracy: 0.8287 - auc: 0.9082 - student_loss: 0.3964 - distillation_loss: 0.0180
Epoch 5/50
192/192 [==============================] - 3s 14ms/step - categorical_accuracy: 0.8420 - auc: 0.9169 - student_loss: 0.3721 - distillation_loss: 0.0154
Epoch 6/50
192/192 [==============================] - 3s 15ms/step - categorical_accuracy: 0.8499 - auc: 0.9231 - student_loss: 0.3580 - distillation_loss: 0.0141
Epoch 7/50
192/192 [==

In [ ]:
# save model Knowlege distillation
student.save(path_result +"KD2_1gram.h5")